In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd

In [ ]:
# Fonction pour initialiser le driver Selenium
def init_driver():
    # Initialize the Edge driver
    service = Service(EdgeChromiumDriverManager().install())
    options = webdriver.EdgeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")  # Cache Selenium
    options.add_argument('--headless')  # Exécute le navigateur en arrière-plan
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")  # Facultatif : exécute en arrière-plan

    driver = webdriver.Edge(service=service, options=options)
    
    return driver

In [3]:
# Fonction pour extraire les liens de la page courante
def get_link_annonces(driver): 
    # Liste des liens d'annonces
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ListItem_title__ndA4s")))
    annonces = driver.find_elements(By.CSS_SELECTOR, "a.ListItem_title__ndA4s")
    liens = [annonce.get_attribute("href") for annonce in annonces]
    return liens

In [4]:
# Fonction pour accepter le popup de consentement
def accept_popup(driver):
    try:
        # Attendre que le popup de consentement apparaisse et l'accepter'
        consent_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "as24-cmp-popup")))
        accept_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='as24-cmp-accept-all-button']")))
        accept_button.click()
        #Attendre que le popup de consentement disparaisse
        WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.ID, "as24-cmp-popup"))) 
    except:
        print("Popup de consentement non trouvé ou déjà fermé.")

In [ ]:
# Fonction pour extraire les informations d'une annonce
def get_info_annonce(driver):
    time.sleep(5)  # Attendre que la page se charge
    data  = {"scraped_at": datetime.today().strftime('%Y-%m-%d')}

    try:
        data["modele"] = driver.find_element(By.CSS_SELECTOR, 'span.StageTitle_boldClassifiedInfo__sQb0l').text.strip()
    except:
        data["modele"] = ''

    try:
        data["prix"] = driver.find_element(By.CSS_SELECTOR, 'span.PriceInfo_price__XU0aF').text.strip()
    except:
        data["prix"] = ''

    try:
        data["finition"] = driver.find_element(By.CSS_SELECTOR, 'div.StageTitle_modelVersion__Yof2Z').text.strip()
    except:
        data["finition"] = ''

    try:
        data["adresse"] = driver.find_element(By.CSS_SELECTOR, 'a.LocationWithPin_locationItem__tK1m5').text.strip()
    except:
        data["adresse"] = ''

    try:
        data["vendeur_name"] = driver.find_element(By.CLASS_NAME, 'TieredPricingRatingsSection_nameContainer__fMSj2').text.strip()
    except:
        data["vendeur_name"] = ''

    try:
        data["price_label"] = driver.find_element(By.CLASS_NAME, 'scr-price-label').text.strip()
    except:
        data["price_label"] = ''

    try:
        data["evaluation_vendeur"] = driver.find_element(By.CLASS_NAME, "scr-visually-hidden").text.strip()
    except:
        data["evaluation_vendeur"] = ''
    
    # Info techniques basiques
    basic_items = driver.find_elements(By.CLASS_NAME, 'VehicleOverview_itemContainer__XSLWi')
    for basic_item in basic_items:
        try:
            label = basic_item.find_element(By.CLASS_NAME, 'VehicleOverview_itemTitle__S2_lb').text.strip().lower()
            value = basic_item.find_element(By.CLASS_NAME, 'VehicleOverview_itemText__AI4dA').text.strip()
            if "kilométrage" in label:
                data['kilometrage'] = value
            elif "transmission" in label:
                data['transmission'] = value
            elif "année" in label:
                data['annee'] = value
            elif "carburant" in label:
                data['carburant'] = value
            elif "puissance" in label:
                data['puissance'] = value
            elif "vendeur" in label:
                data['type_vendeur'] = value
        except:
            continue 
        
    # Info complémentaires
    add_infos = driver.find_elements(By.CLASS_NAME, 'DataGrid_defaultDlStyle__xlLi_')
    for add_info in add_infos:
        try:
            label = add_info.find_element(By.CLASS_NAME, 'DataGrid_defaultDtStyle__soJ6R').text.strip().lower()
            value = add_info.find_element(By.CLASS_NAME, 'DataGrid_defaultDdStyle__3IYpG').text.strip()
            if "carrosserie" in label:
                data['carrosserie'] = value
            elif "sièges" in label:
                data['nb_places'] = value
            elif "portes" in label:
                data['nb_portes'] = value
            elif "émissions de co2" in label:
                data['emission'] = value
            elif "couleur extérieure" in label:
                data['couleur_exterieure'] = value
        except:
            continue
    
    return data

In [1]:
# Fonction principale de scraping autoscout24
def scraping_autoscout24(base_url, max_pages, csv_path):
    ''' 
    Scraping des annonces sur le site autoscout24.fr 
    
    '''
    # Initialiser le driver Edge
    driver = init_driver()

    # Ouvrir la page avec Selenium
    driver.get(base_url)
    time.sleep(5)  # Attendre que la page se charge
    all_data = []
    current_page = 1

    # Accepter le popup de consentement
    accept_popup(driver)

    # Boucle pour parcourir toutes les pages et toutes les annonces
    while True and current_page <= max_pages: 
        print(f"Page {current_page}")

        # Récupérer toutes les annonces de la page
        list_link_annonces = get_link_annonces(driver)
        
        # Collecter les données de chaque annonce
        for annonce in list_link_annonces: 

            # Ouvrir l'annonce dans une nouvelle fenêtre
            driver.execute_script("window.open('"+annonce+"');")
            driver.switch_to.window(driver.window_handles[-1])  # Changer de fenêtre qui correspond à celle de l'annonce
            
            data = get_info_annonce(driver)
            all_data.append(data)

            # Fermer l’onglet et revenir à la page principale
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)
     
        # Accepter le popup de consentement à nouveau au cas où il réapparaît
        accept_popup(driver)
         # Cliquer sur le bouton "Suivant" pour aller à la page suivante
        try:
            next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//li[@class= "prev-next"]/button[@aria-label="Aller à la page suivante"]')))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            next_button.click()
            current_page += 1
        except:
            print("Dernière page atteinte.")
            break

    # Fermer le driver
    driver.quit()

    # Enregistrer les données dans un dataframe pandas
    df = pd.DataFrame(all_data)

    # Exporter les données dans un fichier CSV
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"{len(df)} annonces enregistrées dans le fichier csv")
    print("Fin du scraping !")

    return df

In [7]:
df = scraping_autoscout24("https://www.autoscout24.fr/lst?atype=C&cy=F&desc=0&fregfrom=2015&kmfrom=5000&kmto=100000&offer=U", 10, "../data/raw_data/autoscout24_annonces.csv")

Page 1
Popup de consentement non trouvé ou déjà fermé.
Page 2
Popup de consentement non trouvé ou déjà fermé.
Page 3
Popup de consentement non trouvé ou déjà fermé.
Page 4
Popup de consentement non trouvé ou déjà fermé.
Page 5
Popup de consentement non trouvé ou déjà fermé.
Page 6
Popup de consentement non trouvé ou déjà fermé.
Page 7
Popup de consentement non trouvé ou déjà fermé.
Page 8
Popup de consentement non trouvé ou déjà fermé.
Page 9
Popup de consentement non trouvé ou déjà fermé.
Page 10
Popup de consentement non trouvé ou déjà fermé.
200 annonces enregistrées dans le fichier csv
Fin du scraping !


In [8]:
df.shape

(200, 16)

In [9]:
df.head()

,scraped_at,modele,prix,finition,adresse,vendeur_name,price_label,evaluation_vendeur,kilometrage,transmission,annee,carburant,puissance,type_vendeur,carrosserie,couleur_exterieure
0,2025-04-07,Audi A1,€ 11 990,1.0 TFSI 95 ULTRA ACTIVE S-TRONIC,"Toulouse, FR",Ewigo Toulouse,Pas d'information,Évaluation par étoiles 2.5 sur 5,85 990 km,Boîte automatique,03/2016,Essence,70 kW (95 CH),Pro,Citadine,Gris
1,2025-04-07,Peugeot 308,€ 7 699,II 1.2 PureTech 110 Allure - Entretien constru...,"PARIS, FR",CapCar,Bon prix,Évaluation par étoiles 4 sur 5,90 210 km,Boîte manuelle,06/2016,Essence,81 kW (110 CH),Pro,Berline,Blanc
2,2025-04-07,Citroen C3,€ 5 489,Citroën GENERATION-II 1.2 PURETECH 80 FEEL EDI...,"ANDREZIEUX-BOUTHEON, FR",Ewigo Saint-Etienne,Très bon prix,Évaluation par étoiles 0 sur 5,49 500 km,Boîte manuelle,09/2016,Essence,61 kW (83 CH),Pro,Berline,Gris
3,2025-04-07,Opel Grandland X,€ 10 990,1.2 ECOTEC T 130 INNOVATION 4X2,"Palaiseau, FR",Agence Ewigo Palaiseau,Très bon prix,Évaluation par étoiles 0 sur 5,90 500 km,Boîte manuelle,03/2018,Essence,97 kW (132 CH),Pro,SUV/4x4/Pick-Up,Blanc
4,2025-04-07,Kia Stonic,€ 11 990,1.0 T-GDI 120 DESIGN ISG,"Amiens, FR",Ewigo Amiens,Très bon prix,Évaluation par étoiles 0 sur 5,73 000 km,Boîte manuelle,08/2019,Essence,89 kW (121 CH),Pro,SUV/4x4/Pick-Up,Jaune
